In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Printf
using JLD2

  Activating environment at `~/devel/HW1_S23_solutions/lts_notebooks/Project.toml`


## Q2 (20 pts): Augmented Lagrangian Quadratic Program Solver

Here we are going to use the augmented lagrangian method described [here in a video](https://www.youtube.com/watch?v=0x0JD5uO_ZQ), with [the corresponding pdf here](https://github.com/Optimal-Control-16-745/lecture-notebooks-2022/blob/main/misc/AL_tutorial.pdf) to solve the following problem:

$$\begin{align}
\min_x \quad & \frac{1}{2}x^TQx + q^Tx \\ 
\mbox{s.t.}\quad &  Ax -b = 0 \\ 
&  Gx - h \leq 0 
\end{align}$$
where the cost function is described by $Q \in \mathbb{R}^{n \times n}$, $q \in \mathbb{R}^n$, an equality constraint is described by $A \in \mathbb{R}^{m \times n}$ and $b \in \mathbb{R}^m$, and an inequality constraint is described by $G \in \mathbb{R}^{p \times n}$ and $h \in \mathbb{R}^p$.


By introducing a dual variable $\lambda \in \mathbb{R}^m$ for the equality constraint, and $\mu \in \mathbb{R}^p$ for the inequality constraint, we have the following KKT conditions for optimality:

$$\begin{align}
Qx + q + A^T\lambda + G^T \mu &= 0 \quad \quad \text{stationarity}\\ 
Ax-b&= 0 \quad \quad \text{primal feasibility} \\ 
Gx-h&\leq 0 \quad \quad \text{primal feasibility} \\ 
\mu &\geq 0 \quad \quad \text{dual feasibility} \\ 
\mu \circ (Gx - h) &= 0 \quad \quad \text{complementarity}
  \end{align}$$
  where $\circ$ is element-wise multiplication.  

In [2]:
"""
The data for the QP is stored in `qp` the following way:
    @load joinpath(@__DIR__, "qp_data.jld2") qp 

which is a NamedTuple, where
    Q, q, A, b, G, h = qp.Q, qp.q, qp.A, qp.b, qp.G, qp.h

contains all of the problem data you will need for the QP.

Your job is to make the following function 
    
    x, λ, μ = solve_qp(qp)

You can use (or not use) any of the additional functions:
"""
function cost(qp::NamedTuple, x::Vector)::Real
    0.5*x'*qp.Q*x + dot(qp.q,x)
end
function c_eq(qp::NamedTuple, x::Vector)::Vector
    qp.A*x - qp.b 
end
function h_ineq(qp::NamedTuple, x::Vector)::Vector
    qp.G*x - qp.h
end

function mask_matrix(qp::NamedTuple, x::Vector, μ::Vector, ρ::Real)::Matrix
    error("not implemented")
end
function augmented_lagrangian(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector, ρ::Real)::Real
    error("not implemented")
end
function logging(qp::NamedTuple, main_iter::Int, AL_gradient::Vector, x::Vector, λ::Vector, μ::Vector, ρ::Real)
    # TODO: stationarity norm
    stationarity_norm = 0.0
    @printf("%3d  % 7.2e  % 7.2e  % 7.2e  % 7.2e  % 7.2e  %5.0e\n",
          main_iter, stationarity_norm, norm(AL_gradient), maximum(h_ineq(qp,x)),
          norm(c_eq(qp,x),Inf), abs(dot(μ,h_ineq(qp,x))), ρ)
end
function solve_qp(qp)
    x = zeros(length(qp.q))
    λ = zeros(length(qp.b))
    μ = zeros(length(qp.h))
    
    @printf "iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ\n"
    @printf "----------------------------------------------------------------\n"
    
    for main_iter = 1:20  
        logging(qp, main_iter, zeros(1), x, λ, μ, 0.0)
    end
    return x, λ, μ
end
let 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp)
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  2   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  3   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  4   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  5   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  6   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  7   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  8   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  9   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
 10   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
 11   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
 12   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
 13   0.00e+00   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
 14   0.00e+00

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0])

In [3]:
"""
The data for the QP is stored in `qp` the following way:
    @load joinpath(@__DIR__, "qp_data.jld2") qp 

which is a NamedTuple, where
    Q, q, A, b, G, h = qp.Q, qp.q, qp.A, qp.b, qp.G, qp.h

contains all of the problem data you will need for the QP.

Your job is to make the following function 
    
    x, λ, μ = solve_qp(qp)

You can use (or not use) any of the following functions:
"""
function cost(qp::NamedTuple, x::Vector)::Real
    0.5*x'*qp.Q*x + dot(qp.q,x)
end
function c_eq(qp::NamedTuple, x::Vector)::Vector
    qp.A*x - qp.b 
end
function h_ineq(qp::NamedTuple, x::Vector)::Vector
    qp.G*x - qp.h
end
function kkt_conditions(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector)::Vector
    return [
        qp.A'*λ + qp.G'*μ + qp.Q*x + qp.q; # stationarity
        c_eq(qp,x);
        (h_ineq(qp,x) .* μ)
    ]
end
function mask_matrix(qp::NamedTuple, x::Vector, μ::Vector, ρ::Real)::Matrix
    h = h_ineq(qp,x)
    Iρ = zeros(length(μ), length(μ))
    for i = 1:length(μ)
        if ((h[i] < 0) && (μ[i] ==0))
            Iρ[i,i]=0
        else
            Iρ[i,i]=ρ
        end
    end
    return Iρ
end
function augmented_lagrangian(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector, ρ::Real)::Real
    h = h_ineq(qp,x)
    c = c_eq(qp,x)
    Iρ = mask_matrix(qp, x, μ, ρ)
    cost(qp,x) + λ'*c + μ'*h + 0.5*ρ*c'c + 0.5*h'*Iρ*h
end
function logging(qp, main_iter, al_gradient, x, λ, μ, ρ)
    stationarity = norm(qp.A'*λ + qp.G'*μ + qp.Q*x + qp.q)
    @printf("%3d  % 7.2e  % 7.2e  % 7.2e  % 7.2e  % 7.2e  %5.0e\n",
          main_iter, stationarity, norm(al_gradient), maximum(h_ineq(qp,x)),
          norm(c_eq(qp,x),Inf), abs(dot(μ,h_ineq(qp,x))), ρ)
end
function solve_qp(qp)
    x = zeros(length(qp.q))
    λ = zeros(length(qp.b))
    μ = zeros(length(qp.h))
    ρ = 1.0 
    ϕ = 10.0
    
    @printf "iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ\n"
    @printf "----------------------------------------------------------------\n"
    for main_iter = 1:20
        g = FD.gradient(_x->augmented_lagrangian(qp,_x,λ,μ,ρ), x)
        if norm(g) < 1e-6 
            λ += ρ*c_eq(qp, x)
            μ = max.(0, μ + ρ*h_ineq(qp,x))
            ρ *= ϕ
        else
            H = FD.hessian(_x->augmented_lagrangian(qp,_x,λ,μ,ρ), x)
            x += -H\g
        end
        logging(qp, main_iter, g, x, λ, μ, ρ)
        
        if (maximum(h_ineq(qp,x)) < 1e-8) && (norm(c_eq(qp,x),Inf) < 1e-8)
            @info "success"
            break
        end  
    end
    
    return x, λ, μ
end

solve_qp (generic function with 1 method)

In [4]:
using Test 
@testset "qp solver" begin 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp)
    
    @load joinpath(@__DIR__, "qp_solutions.jld2") qp_solutions
    @test norm(x - qp_solutions.x,Inf)<1e-3;
    @test norm(λ - qp_solutions.λ,Inf)<1e-3;
    @test norm(μ - qp_solutions.μ,Inf)<1e-3;
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   0.00e+00   5.60e+01   1.55e+00   1.31e+00   0.00e+00  1e+00
  2   0.00e+00   4.83e+00   5.51e-01   1.27e+00   0.00e+00  1e+00
  3   0.00e+00   5.25e-15   5.51e-01   1.27e+00   4.59e-01  1e+01
  4   0.00e+00   4.92e+01   2.56e-02   3.07e-01   4.94e-02  1e+01
  5   0.00e+00   3.32e-14   2.56e-02   3.07e-01   1.05e-02  1e+02
  6   0.00e+00   8.87e+01   6.84e-03   1.35e-02   4.55e-04  1e+02
  7   0.00e+00   2.68e-13   6.84e-03   1.35e-02   7.94e-03  1e+03
  8   0.00e+00   4.28e+01   6.84e-02   1.55e-04   1.40e-04  1e+03
  9   0.00e+00   2.13e+02   3.64e-05   1.62e-04   1.17e-04  1e+03
 10   0.00e+00   3.12e-12   3.64e-05   1.62e-04   1.06e-04  1e+04
 11   0.00e+00   5.30e+00  -5.61e-09   2.05e-08   1.14e-08  1e+04
 12   0.00e+00   3.11e-11  -5.61e-09   2.05e-08   1.14e-08  1e+05
 13   0.00e+00   5.21e-03  -1.57e-13   3.92e-13   1.88e-13  1e+05


[ Info: success


Test Summary: | Pass  Total
qp solver     |    3      3


Test.DefaultTestSet("qp solver", Any[], 3, false, false)

# Simulating a Falling Brick with QP's
In this question we'll be simulating a brick falling and sliding on ice in 2D. You will show that this problem can be formulated as a QP, which you will solve using an Augmented Lagrangian method.

## The Dynamics
The dynamics of the brick can be written in continuous time as
$$ M \dot{v}  + M g = J^T \lambda \\ \text{ where } M = mI, \; g = \begin{bmatrix} 0 \\ 9.81 \end{bmatrix},\; J = \begin{bmatrix} 0 & 1 \end{bmatrix} $$
and $\lambda \in \mathbb{R}$ is the normal force. The velocity $v \in \mathbb{R}^2$ and position $q \in \mathbb{R}^2$ are composed of the horizontal and vertical components.

We can discretize the dynamics with backward Euler:
$$ \begin{align} \begin{bmatrix} v_{k+1} \\ q_{k+1} \end{bmatrix} = \begin{bmatrix} v_k \\ q_k \end{bmatrix}
+ h \begin{bmatrix} \frac{1}{m} J^T \lambda_{k+1} - g \\ v_{k+1} \end{bmatrix} \end{align} $$

We also have the following contact constraints:
$$ \begin{align}
J q_{k+1} &\geq 0 &&\text{(don't fall through the ice)} \\
\lambda_{k+1} &\geq 0 &&\text{(normal forces only push, not pull)} \\
\lambda_{k+1} J q_{k+1} &= 0 &&\text{(no force at a distance)}
\end{align} $$

## Part (a): QP formulation (5 pts)
Show that these discrete-time dynamics are equivalent to the following QP by writing down the KKT conditions.

$$ \begin{align}
    &\text{minimize}_{v_{k+1}} && \frac{1}{2} v_{k+1}^T M v_{k+1} + v_{k+1}^T M (hg - v_k) \\
    &\text{subject to} && -J(q_k + h v_{k+1}) \leq 0 \\
\end{align} $$

In [ ]:
# add brick sim 

In [13]:
# now let's solve a actuator allocation problem 
using Random
using LinearAlgebra
let 
    
    rng = MersenneTwister(1234)
    nu = 10
    
    u_max = 0.5*ones(nu)
    u_min = -0.3*ones(nu)
    u_desired = clamp.(0.5*randn(rng, nu), u_min, u_max)
    @show u_desired
    
#     A = randn(rng, 3, nu)
#     b = randn(rng, 3)
#     Q = randn(rng, nu, nu); Q = Q'*Q + I 
#     q = 10*randn(rng, nu)
    
#     G = [diagm(ones(nu));-diagm(ones(nu))]
#     h = [u_max; -u_min]
    
#     qp = (Q = Q, q = q, A = A, b = b, G=G, h = h)
    
#     u, λ, μ = solve_qp(qp)
#     @show u 

    
end

u_desired = [0.4336736009756228, -0.3, -0.24723937675211696, -0.3, 0.4322006566267577, 0.5, 0.2664066410847691, -0.13586769801731033, 0.2511672481943337, -0.2584918103466343]


10-element Vector{Float64}:
  0.4336736009756228
 -0.3
 -0.24723937675211696
 -0.3
  0.4322006566267577
  0.5
  0.2664066410847691
 -0.13586769801731033
  0.2511672481943337
 -0.2584918103466343